In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

test_df: pd.DataFrame = pd.read_pickle('out/test.pkl')

test_df = test_df.groupby('id').apply(lambda x: pd.Series({
    'history': x['history'].iloc[0],
    'positives': set(x['ID'])
}), include_groups=False)
test_df

,history,positives
id,,
1005_2024-11-06,"[[26, 0], [5, 1], [16, 1]]",{6}
1008_2024-11-04,"[[2, 1], [28, 0], [18, 1]]",{6}
1008_2024-11-05,"[[2, 1], [28, 0], [18, 1], [28, 0], [6, 1], [1...","{10, 30, 14}"
1008_2024-11-06,"[[2, 1], [28, 0], [18, 1], [28, 0], [6, 1], [1...","{0, 18, 11}"
1008_2024-11-07,"[[28, 0], [6, 1], [19, 0], [30, 1], [14, 1], [...","{5, 15}"
...,...,...
957_2024-11-06,"[[1, 0], [9, 0], [19, 0], [22, 0], [0, 0], [7,...",{2}
957_2024-11-07,"[[22, 0], [0, 0], [7, 0], [21, 0], [15, 0], [2...",{2}
972_2024-11-01,"[[21, 0], [17, 0], [0, 1], [12, 0], [26, 0], [...",{1}


In [ ]:
import torch
import numpy as np
from src.gru4rec import GRU4Rec, DEVICE

model: GRU4Rec = torch.load('models/gru4rec.pth')
model.eval()
missions = pd.read_csv('data/missions.csv')

def rank(x):
    history = torch.from_numpy(x['history']).view(1, -1, 2).to(DEVICE)
    with torch.no_grad():
        scores = model(history).view(-1).detach().cpu().numpy()
    return np.argsort(scores)[::-1]

def random_rank(_):
    return np.random.permutation(len(missions))

test_df['gru4rec'] = test_df.progress_apply(rank, axis=1)
test_df['random'] = test_df.progress_apply(random_rank, axis=1)
test_df

/tmp/ipykernel_23968/133253311.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model: GRU4Rec = torch.load('models/gru4rec.pth')


  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

,history,positives,gru4rec,random
id,,,,
1005_2024-11-06,"[[26, 0], [5, 1], [16, 1]]",{6},"[30, 31, 4, 25, 2, 5, 13, 0, 6, 24, 1, 26, 7, ...","[24, 12, 18, 10, 3, 8, 19, 9, 30, 16, 11, 14, ..."
1008_2024-11-04,"[[2, 1], [28, 0], [18, 1]]",{6},"[31, 30, 2, 4, 0, 25, 6, 27, 5, 7, 24, 13, 26,...","[7, 0, 21, 19, 8, 20, 13, 4, 10, 26, 18, 17, 1..."
1008_2024-11-05,"[[2, 1], [28, 0], [18, 1], [28, 0], [6, 1], [1...","{10, 30, 14}","[2, 30, 31, 4, 0, 5, 24, 12, 25, 27, 6, 7, 26,...","[24, 1, 13, 8, 4, 16, 18, 2, 15, 0, 21, 19, 7,..."
1008_2024-11-06,"[[2, 1], [28, 0], [18, 1], [28, 0], [6, 1], [1...","{0, 18, 11}","[31, 2, 4, 30, 5, 0, 24, 25, 12, 26, 6, 7, 27,...","[10, 8, 28, 9, 15, 18, 24, 26, 20, 21, 25, 11,..."
1008_2024-11-07,"[[28, 0], [6, 1], [19, 0], [30, 1], [14, 1], [...","{5, 15}","[31, 30, 4, 2, 0, 5, 25, 12, 14, 7, 24, 13, 26...","[27, 17, 3, 4, 13, 16, 20, 8, 2, 10, 22, 6, 0,..."
...,...,...,...,...
957_2024-11-06,"[[1, 0], [9, 0], [19, 0], [22, 0], [0, 0], [7,...",{2},"[30, 2, 31, 4, 5, 0, 24, 25, 12, 26, 6, 13, 7,...","[15, 6, 8, 27, 25, 17, 29, 22, 30, 28, 7, 23, ..."
957_2024-11-07,"[[22, 0], [0, 0], [7, 0], [21, 0], [15, 0], [2...",{2},"[30, 2, 31, 4, 0, 5, 25, 12, 6, 24, 26, 7, 27,...","[25, 30, 27, 6, 24, 23, 28, 3, 0, 16, 13, 29, ..."
972_2024-11-01,"[[21, 0], [17, 0], [0, 1], [12, 0], [26, 0], [...",{1},"[30, 2, 4, 31, 0, 12, 24, 5, 25, 6, 26, 13, 7,...","[8, 9, 14, 11, 15, 17, 29, 16, 12, 0, 10, 26, ..."


In [3]:
def recall(x, model, k=5):
    top_k = set(x[model][:k])
    return len(x['positives'] & top_k) / len(x['positives'])

def hit_rate(x, model, k=5):
    top_k = set(x[model][:k])
    return len(x['positives'] & top_k) > 0

pd.DataFrame({
    'gru4rec': {
        'recall@5': test_df.progress_apply(lambda x: recall(x, 'gru4rec'), axis=1).mean(),
        'recall@10': test_df.progress_apply(lambda x: recall(x, 'gru4rec', k=10), axis=1).mean(),
        'hit_rate@5': test_df.progress_apply(lambda x: hit_rate(x, 'gru4rec'), axis=1).mean(),
        'hit_rate@10': test_df.progress_apply(lambda x: hit_rate(x, 'gru4rec', k=10), axis=1).mean()
    },
    'random': {
        'recall@5': test_df.progress_apply(lambda x: recall(x, 'random'), axis=1).mean(),
        'recall@10': test_df.progress_apply(lambda x: recall(x, 'random', k=10), axis=1).mean(),
        'hit_rate@5': test_df.progress_apply(lambda x: hit_rate(x, 'random'), axis=1).mean(),
        'hit_rate@10': test_df.progress_apply(lambda x: hit_rate(x, 'random', k=10), axis=1).mean()
    }
})

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

  0%|          | 0/2074 [00:00<?, ?it/s]

,gru4rec,random
recall@5,0.300121,0.165935
recall@10,0.545355,0.314899
hit_rate@5,0.489392,0.314368
hit_rate@10,0.830280,0.528930
